ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- time_seconds


ValueError: The feature names should match those that were passed during fit.
Feature names seen at fit time, yet now missing:
- time_seconds


In [3]:
import pandas as pd
import pickle

# train preprocesss, passed the train csv throgh hte preprocessing.ipynb
test_df = pd.read_csv('test_preprocessed.csv')

# making sure to have the ids with us to match the submission matching the rows
ids = test_df['id']

# ## now the drop of the id and another similar
test_df = test_df.drop(columns=['id', 'p_num'])

#h m s time to the seconds in a cols and drop the initial H:M:S format
def time_to_seconds(time_str):
    h, m, s = map(int, time_str.split(':'))
    return h * 3600 + m * 60 + s

if 'time' in test_df.columns:
    test_df['time_seconds'] = test_df['time'].apply(time_to_seconds)
    test_df = test_df.drop(columns=['time'])  # removing the original H:M:S format column of the time

# pca model saved during the training of the data 
with open('best_model_pca.pkl', 'rb') as pca_file:
    pca = pickle.load(pca_file)

#### features used in PCA 
pca_features = test_df.columns.intersection(pca.feature_names_in_)
test_pca = pca.transform(test_df[pca_features])

# adding the pca components to the original feature
for i in range(test_pca.shape[1]):
    test_df[f'PC{i+1}'] = test_pca[:, i]

#the best model, xgb 
with open('best_model.pkl', 'rb') as model_file:
    model = pickle.load(model_file)

### lets make sure that the train_prepro contains the expected columns in the right order 

train_columns = model.get_booster().feature_names
test_df = test_df.reindex(columns=train_columns, fill_value=0)

### model execution 
predictions = model.predict(test_df)

#submission csv matching the target values to the ids
submission = pd.DataFrame({'id': ids, 'target': predictions})
submission.to_csv('submission.csv', index=False)
print("Submission file 'submission.csv' created successfully.")


Submission file 'submission.csv' created successfully.
